In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
import time
import datetime
from pprint import pprint as pp
import sys

sys.path.insert(0, '/home/molly/Desktop/DeepTCGA/')
import load_data

/home/molly/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
def fc_layer(A_prev, size_in, size_out, name="fully-connected"):
    with tf.name_scope(name):
        w = tf.Variable(tf.truncated_normal([size_in, size_out], mean=0, stddev=0.1))
        b = tf.Variable(tf.constant(0.1, shape=[size_out]))
        act = tf.matmul(A_prev, w) + b
        return act, w, b
    

def random_sparse_layer(A_prev, size_in, size_out, sparsity, name="sparse"):
    if sparsity < 0.01:
        mask = generate_really_sparse_mask(size_in, size_out, sparsity)
    else:
        mask = generate_random_sparse_mask(size_in, size_out, sparsity)
    with tf.name_scope(name):
        w = tf.Variable(tf.truncated_normal([size_in, size_out], mean=0, stddev=0.1))
        w_masked = tf.multiply(mask, w)
        b = tf.Variable(tf.constant(0.1, shape=[size_out]))
        act = tf.matmul(A_prev, w_masked) + b
        return act, w, w_masked, mask, b

    
def build_model(x, N_IN, N_HIDDEN, sparsity=0.5):
    parameters = {}
    z1, w1_init, w1_sparse, mask1, b1 = random_sparse_layer(
        x, N_IN, N_HIDDEN, sparsity, name="sparse1")
    parameters.update({"z1":z1, "w1_init": w1_init,"w1_sparse":w1_sparse, 
                       "mask1": mask1, "b1": b1})
    hidden = tf.nn.leaky_relu(z1, name="hidden")
    x_recon, w2, b2 = fc_layer(hidden, N_HIDDEN, N_IN, name="fc1")
    parameters.update({"w2": w2, "b2": b2})
    return x_recon, parameters


def build_model_right_sparse(x, N_IN, N_HIDDEN, sparsity=0.5):
    parameters = {}
    z1, w1, b1 = fc_layer(x, N_IN, N_HIDDEN, name="fc1")
    parameters.update({"w1": w1, "b1": b1})    
    hidden = tf.nn.leaky_relu(z1, name="hidden")
    x_recon, w2_init, w2_sparse, mask2, b2 = random_sparse_layer(
        hidden, N_HIDDEN, N_IN, sparsity, name="sparse1")
    parameters.update({"w2_init": w2_init, "w2_sparse":w2_sparse, 
                       "mask2": mask2, "b2": b2})
    return x_recon, parameters


def build_model_both_sparse_assymetric(x, N_IN, N_HIDDEN, sparsity=0.5):
    parameters = {}
    z1, w1_init, w1_sparse, mask1, b1 = random_sparse_layer(
        x, N_IN, N_HIDDEN, sparsity, name="sparse1")
    parameters.update({"z1":z1, "w1_init": w1_init,"w1_sparse":w1_sparse, 
                       "mask1": mask1, "b1": b1})
    hidden = tf.nn.leaky_relu(z1, name="hidden")
    x_recon, w2_init, w2_sparse, mask2, b2 = random_sparse_layer(
        hidden, N_HIDDEN, N_IN, sparsity, name="sparse2")
    parameters.update({"w2_init": w2_init, "w2_sparse":w2_sparse, 
                       "mask2": mask2, "b2": b2}) 
    return x_recon, parameters


def build_model_both_sparse_symetric(x, N_IN, N_HIDDEN, sparsity=0.5):
    parameters = {}
    z1, w1_init, w1_sparse, mask1, b1 = random_sparse_layer(
        x, N_IN, N_HIDDEN, sparsity, name="sparse1")
    parameters.update({"z1":z1, "w1_init": w1_init,"w1_sparse":w1_sparse, 
                       "mask1": mask1, "b1": b1})
    hidden = tf.nn.leaky_relu(z1, name="hidden")
    mask2 =tf.transpose(mask1)
    w2_init = tf.Variable(tf.truncated_normal([N_HIDDEN, N_IN], mean=0, stddev=0.1))
    w2_sparse = tf.multiply(mask2, w2_init)
    b2 = tf.Variable(tf.constant(0.1, shape=[N_IN]))
    x_recon = tf.matmul(hidden, w2_sparse) + b2
    parameters.update({"w2_init": w2_init, "w2_sparse":w2_sparse, 
                       "mask2": mask2, "b2": b2})     
    return x_recon, parameters


def back_prop(x, x_recon, learning_rate):
    loss = tf.reduce_mean(tf.square(x_recon - x))
    train_step = tf.train.AdamOptimizer(learning_rate).minimize(loss)
    return train_step


def feed_forward(x, parameters):
    w1_sparse, b1 = parameters["w1_sparse"], parameters["b1"]
    hidden = tf.nn.leaky_relu(tf.matmul(x, w1_sparse) + b1)
    w2, b2 = parameters["w2"], parameters["b2"]
    x_recon = tf.matmul(hidden, w2) + b2
    return x_recon, hidden


def feed_forward_right_sparse(x, parameters):
    w1, b1 = parameters["w1"], parameters["b1"]
    hidden = tf.nn.leaky_relu(tf.matmul(x, w1) + b1)
    w2, b2 = parameters["w2_sparse"] ,parameters["b2"]
    x_recon = tf.matmul(hidden, w2) + b2
    return x_recon, hidden


def feed_forward_both_sparse(x, parameters):
    w1, b1 = parameters["w1_sparse"], parameters["b1"]
    hidden = tf.nn.leaky_relu(tf.matmul(x, w1) + b1)
    w2, b2 = parameters["w2_sparse"] ,parameters["b2"]
    x_recon = tf.matmul(hidden, w2) + b2
    return x_recon, hidden

def mse(x, x_recon, name=""):
    mse = tf.reduce_mean(tf.square(x_recon-x))
    mse_summary = tf.summary.scalar(name + "_mse", mse)
    return mse, mse_summary


def generate_random_sparse_mask(size_in, size_out, sparsity):
    """ make sure each node has at least one edge connecting to it"""
    assert(size_out>=100)
    while True:
        try:
            a = np.random.uniform(low=0, high=1, size=(size_in, size_out))
            a = (a>(1-sparsity)).astype(int)
            assert((a.sum(axis=1)>1).all())
            assert((a.sum(axis=0)>1).all())
            break
        except AssertionError:
            continue
    return tf.constant(a.astype('float32'))


def generate_really_sparse_mask(size_in, size_out, sparsity):
    a = np.random.uniform(low=0, high=1, size=(size_in, size_out))
    a = (a>(1-sparsity)).astype(int)
    return tf.constant(a.astype('float32'))

In [3]:
def train_model(data, batch_size=128, n_hidden=1000, 
                num_epoch=1000, learning_rate=1e-3, sparsity=1, mode="left", extra=""):
    tf.reset_default_graph()
    LOGDIR = "/tmp/tcga_{0}".format(datetime.datetime.today().date())
    N_IN = data.train.num_features
    N_OUT = data.train.num_features
    N_HIDDEN = n_hidden
    
    # train step
    (train_batch, train_iter, val_all, val_iter, 
        train_all, train_iter_all) = data.prep_batch(batch_size=batch_size, 
                                                     count_by="epoch")
    x = train_batch["X"]
    if mode=="left":
        x_recon, parameters = build_model(x, N_IN, N_HIDDEN, sparsity=sparsity)
    elif mode == "both":
        x_recon, parameters = build_model_both_sparse_symetric(x, N_IN, N_HIDDEN, sparsity=sparsity)        
    else:
        raise
    train_step = back_prop(x, x_recon, learning_rate)    
    
    # mse
    x_train, x_val = train_all["X"], val_all["X"]
    if mode=="left":
        x_train_recon, train_latent = feed_forward(x_train, parameters)
        x_val_recon, val_latent = feed_forward(x_val, parameters)
    elif mode == "both":
        x_train_recon, train_latent = feed_forward_both_sparse(x_train, parameters)
        x_val_recon, val_latent = feed_forward_both_sparse(x_val, parameters)
    
    train_mse, train_summ = mse(x_train, x_train_recon, name="train")
    val_mse, val_summ = mse(x_val, x_val_recon, name="valiation")
    
    # run
    sess = tf.Session()
    summ = tf.summary.merge_all()
    writer = tf.summary.FileWriter(LOGDIR + "ae_{0}".format(extra))
    writer.add_graph(sess.graph)                            
    sess.run(tf.global_variables_initializer())

    for epoch in range(num_epoch):
        sess.run([train_iter.initializer])
        t0 = time.time()
        try:
            while True:
                sess.run(train_step)
        except tf.errors.OutOfRangeError:
            sess.run([train_iter_all.initializer, val_iter.initializer])
            [train_error, train_s, val_error, val_s] = sess.run(
                [train_mse, train_summ, val_mse, val_summ])
            writer.add_summary(train_s, epoch)
            writer.add_summary(val_s, epoch)
            if epoch % 10 == 0:
                print("epoch", epoch)
                print("training mse:", train_error, "validation mse", val_error)
                print("epoch time:", time.time()-t0)
#     train_latent = sess.run(train_latent)
#     val_latent = sess.run(val_latent)
#     np.save("./results/AE/train_latent_sparse2.npy", train_latent)
#     np.save("./results/AE/val_latent_sparse2.npy", val_latent)    
    sess.close()

In [4]:
tcga = load_data.read_data_sets("../data/mRNA_lognorm_StandardScaled.hdf")

In [33]:
for sparsity in [0.001]:
    for mode in ["both"]:
        train_model(tcga, n_hidden=5000, num_epoch=1000, mode=mode, sparsity=sparsity,
            extra="5000_hidden_{0}_sparsity_{1}".format(mode, sparsity))

ResourceExhaustedError: OOM when allocating tensor with shape[2147,16335] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[Node: MatMul_4 = MatMul[T=DT_FLOAT, transpose_a=false, transpose_b=false, _device="/job:localhost/replica:0/task:0/device:GPU:0"](LeakyRelu_1/Maximum, Mul)]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.


Caused by op 'MatMul_4', defined at:
  File "/home/molly/anaconda3/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/home/molly/anaconda3/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/home/molly/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/home/molly/anaconda3/lib/python3.6/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start